In [1]:
import pandas as pd
import numpy as np
import requests
import json
import requests
import utils as ut

# Para traer la api key
import os
from dotenv import load_dotenv

In [2]:
# Importamos la api key
# Obtén la ruta absoluta del archivo .env
env_path = os.path.join(os.path.dirname('testing.ipynb'), '..', 'conf', '.env')

# Carga las variables de entorno desde el archivo .env
load_dotenv(env_path)

# Accede a la variable de entorno
key_admin = os.getenv('key_admin')

---

In [3]:
id = ut.datos_invocador(['GC Tincho'])['id']
id

Se finalizo el proceso de transformacion para GC Tincho.


0    y3gAnTO-EtujHRBDa8IOytuTLb1b_mq7f-MIqIarQEoBvQ
Name: id, dtype: object

In [4]:
def datos_invocador(nombres_invocador:list)->pd.DataFrame:
    
    # Definimos algunas variables
    key = key_admin
    region = 'LA2'
    
    # Creamos la variable para almacenar 
    df_personal_infomation = pd.DataFrame()
    
    # Creamos un bucle for para que funcionen la lista de nombres a emplear.
    for nombre_invocador in nombres_invocador:
        
        # Conectamos a la API
        url = f'https://{region}.api.riotgames.com/lol/summoner/v4/summoners/by-name/{nombre_invocador}?api_key={key}' 
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Error: no se pudo obtener la maestría de campeones del invocador {nombre_invocador}.")
            return None

        # Lo transformamos en un texto
        texto = response.text

        # Lo transformamos en un JSON y lo normalizamos en un pd.df
        json_info = json.loads(texto)
        personal_infomation = pd.json_normalize(json_info)

        # Almacenamos los datos
        df_personal_infomation = pd.concat([df_personal_infomation, personal_infomation], ignore_index=True)

        print(f'Se finalizo el proceso de transformacion para {nombre_invocador}.')
    
    return df_personal_infomation

In [21]:
# Creamos la funcion para que nos de los datos de la partida en viv
    
def live_match(nombre_jugador:str)->pd.DataFrame:
    
    #definimos algunas variables
    key = key_admin
    region = 'la2'
        
    # Creamos el df para almacenar los datos
    df_match = pd.DataFrame()
    
    # Obtenemos el id del jugador
    id = datos_invocador([nombre_jugador])['id'][0]
    print(f'El id del jugador es: {id}')
    
    # Conexion a la API
    url = f'https://{region}.api.riotgames.com/lol/spectator/v4/active-games/by-summoner/{id}?api_key={key_admin}'
    response = requests.get(url)
    if response.status_code != 200:
        print(f'Error al obtener datos de: {nombre_jugador}')
        return None
    
    # Lo transformamos en texto 
    texto = response.text
    
    # Lo transformamos en JSON y normalziamos con pd
    json_info = json.loads(texto)
    df_match = pd.json_normalize(json_info)
    
    print(f'Se finalizo el proceso para la partida de {nombre_jugador}')
    
        # Creamos el df vacio
    detalle_partida = pd.DataFrame()

    # Creamos el bucle para normalizar cada JSON    
    for i in range(0,10,1):
        
        # Normalizamos cada jugador
        player_normalize = pd.json_normalize(pd.json_normalize(df_match['participants'])[i])
        
        # Guardamos los datos
        detalle_partida = pd.concat([detalle_partida, player_normalize])

    return detalle_partida

In [22]:
partida = live_match('GC Crotolamo')
partida

# pd.json_normalize(pd.json_normalize(partida['participants'])[0])

Se finalizo el proceso de transformacion para GC Crotolamo.
El id del jugador es: 500-z1qVQ8GwiEHubwYokUDR5dhy5OSePjRTNBdnm3OK6w
Se finalizo el proceso para la partida de GC Crotolamo


,teamId,spell1Id,spell2Id,championId,profileIconId,summonerName,bot,summonerId,gameCustomizationObjects,perks.perkIds,perks.perkStyle,perks.perkSubStyle
0,100,4,14,99,5736,Foortune,False,55MEpu53V7U8jG3j_umw8AuBPQAGpaj7EA7D75inMfyGxqM,[],"[8229, 8226, 8210, 8237, 8126, 8106, 5008, 500...",8200,8100
0,100,4,12,150,4864,M1KE TYS0N,False,FP95A4NcwZsbHYKATDFJpAMv8wzOpyxHxnfkadid94XNeCQ,[],"[8010, 9111, 9104, 8299, 8473, 8451, 5005, 500...",8000,8400
0,100,4,14,518,5434,AntonyKiller77,False,VxqE0JuZi21SybY9oHea7IRvliueUMhz4sKn5cTZeCDDS2...,[],"[8112, 8139, 8138, 8135, 8226, 8237, 5005, 500...",8100,8200
0,100,4,11,121,5755,PermAcànc3r,False,nU5_o77OIGTyHeMPNLC2mQ1L3khgtloykhzXa4HAuGHwuf...,[],"[8369, 8304, 8321, 8347, 8143, 8135, 5008, 500...",8300,8100
0,100,7,4,81,11,peneriko123,False,8s9RcWQHEFrV4BjRNloqMVa8I5LCSGknzuzTuCM_cp64YS...,[],"[8005, 8009, 9103, 8014, 8345, 8304, 5005, 500...",8000,8300
0,200,4,3,143,5840,Ptrick,False,Jzh7TLAzGb2dg5uYZdqUpvUHqTx114WHRb4M19s01oLWGb...,[],"[8229, 8226, 8210, 8237, 8345, 8347, 5008, 500...",8200,8300
0,200,4,11,111,986,GC Crotolamo,False,500-z1qVQ8GwiEHubwYokUDR5dhy5OSePjRTNBdnm3OK6w,[],"[8439, 8401, 8473, 8242, 8345, 8347, 5007, 500...",8400,8300
0,200,12,4,126,3501,GC Ca Cdos 601,False,0nPz7ri619KkdYlJiPSnsVS35NgxoQtJoWH_2wcjYRSLRA,[],"[8230, 8226, 8210, 8236, 8009, 8014, 5005, 500...",8200,8000
0,200,7,4,22,5031,Magnuskid,False,8HoShGdX3uGMwj5mMr2uLDcnd7etdDMK4o4UX6Ut0Keu,[],"[8008, 9111, 9104, 8014, 8126, 8106, 5005, 500...",8000,8100
0,200,4,14,17,5840,Tiburoncionuhaha,False,CtbCzfJHIBPDKOOpTSEbREyrISbmP8F_dXVKVtOxwtHv,[],"[8437, 8446, 8473, 8451, 8126, 8106, 5005, 500...",8400,8100


In [20]:
# Creamos el df vacio
detalle_partida = pd.DataFrame()

# Creamos el bucle para normalizar cada JSON    
for i in range(0,10,1):
    
    # Normalizamos cada jugador
    player_normalize = pd.json_normalize(pd.json_normalize(partida['participants'])[i])
    
    # Guardamos los datos
    detalle_partida = pd.concat([detalle_partida, player_normalize])

    
detalle_partida

,teamId,spell1Id,spell2Id,championId,profileIconId,summonerName,bot,summonerId,gameCustomizationObjects,perks.perkIds,perks.perkStyle,perks.perkSubStyle
0,100,4,14,99,5736,Foortune,False,55MEpu53V7U8jG3j_umw8AuBPQAGpaj7EA7D75inMfyGxqM,[],"[8229, 8226, 8210, 8237, 8126, 8106, 5008, 500...",8200,8100
0,100,4,12,150,4864,M1KE TYS0N,False,FP95A4NcwZsbHYKATDFJpAMv8wzOpyxHxnfkadid94XNeCQ,[],"[8010, 9111, 9104, 8299, 8473, 8451, 5005, 500...",8000,8400
0,100,4,14,518,5434,AntonyKiller77,False,VxqE0JuZi21SybY9oHea7IRvliueUMhz4sKn5cTZeCDDS2...,[],"[8112, 8139, 8138, 8135, 8226, 8237, 5005, 500...",8100,8200
0,100,4,11,121,5755,PermAcànc3r,False,nU5_o77OIGTyHeMPNLC2mQ1L3khgtloykhzXa4HAuGHwuf...,[],"[8369, 8304, 8321, 8347, 8143, 8135, 5008, 500...",8300,8100
0,100,7,4,81,11,peneriko123,False,8s9RcWQHEFrV4BjRNloqMVa8I5LCSGknzuzTuCM_cp64YS...,[],"[8005, 8009, 9103, 8014, 8345, 8304, 5005, 500...",8000,8300
0,200,4,3,143,5840,Ptrick,False,Jzh7TLAzGb2dg5uYZdqUpvUHqTx114WHRb4M19s01oLWGb...,[],"[8229, 8226, 8210, 8237, 8345, 8347, 5008, 500...",8200,8300
0,200,4,11,111,986,GC Crotolamo,False,500-z1qVQ8GwiEHubwYokUDR5dhy5OSePjRTNBdnm3OK6w,[],"[8439, 8401, 8473, 8242, 8345, 8347, 5007, 500...",8400,8300
0,200,12,4,126,3501,GC Ca Cdos 601,False,0nPz7ri619KkdYlJiPSnsVS35NgxoQtJoWH_2wcjYRSLRA,[],"[8230, 8226, 8210, 8236, 8009, 8014, 5005, 500...",8200,8000
0,200,7,4,22,5031,Magnuskid,False,8HoShGdX3uGMwj5mMr2uLDcnd7etdDMK4o4UX6Ut0Keu,[],"[8008, 9111, 9104, 8014, 8126, 8106, 5005, 500...",8000,8100
0,200,4,14,17,5840,Tiburoncionuhaha,False,CtbCzfJHIBPDKOOpTSEbREyrISbmP8F_dXVKVtOxwtHv,[],"[8437, 8446, 8473, 8451, 8126, 8106, 5005, 500...",8400,8100


In [14]:
partida

### __Cosas a considerar de cada partida__
- __gameMode__
- __gameType__
- __gameLength__